In [5]:
# Ranging #
from pypozyx import *
import numpy as np
from pypozyx.definitions.registers import *
import sys
import time
import serial 
from openpyxl import Workbook
import matplotlib.pyplot as plt
                
write_wb = Workbook()
write_ws = write_wb.active

def cal(cira,th):
    idx = 0
    cir_cal = 0
    l = len(cira)
    mag = abs(cira[:-36])
    for i in range(1,993):
        if(mag[i]>th):
            if(mag[i]>mag[i-1] and mag[i]>mag[i+1]):
                idx = i
                break
    tmp = mag[idx]-500
    mag_idx = 0
    for i in range(idx+1,993):
        if(mag[i] > tmp and mag[i] > th):
            mag_idx = i
            tmp = mag[i]
    tmp_idx = mag_idx - idx
    if (mag_idx - idx < 40):
         tmp_idx = 0
    cir_cal= tmp_idx*64*3*pow(10,8)/(499.2*pow(10,6))
    print(mag_idx-idx)
    return cir_cal

def cal2(cira):
    l = len(cira)
    phase = phase(cira[:-36])
    tmp = 0
    cnt = 0
    for i in range(0,l):
        if(mag[i]>th):
            if(mag[i]>mag[i-1] and mag[i]>mag[i+1]):
                idx = i
                break
    for i in range(idx+1,993):
        if(mag[i] > tmp and mag[i] > th):
            mag_idx = i
            tmp = mag[i]
    for i in range(idx+1, mag_idx):
        if(phase[i] > phase[i-1] and phase[i] > phase[i+1]):
            cnt = cnt + 1
    delay = phase[mag_idx] - phase[idx] + cnt * 360
    cir_cal = delay * (3.14/180) * 3*pow(10,8) * (1/(2*3.14*6.24*10^9))
    return cir_cal

i = 1    
port = '/dev/cu.usbmodem328B345A30371'
p = PozyxSerial(port)

remote_id = 0x696a          # the network ID of the remote device
remote = True               # whether to use the given remote device for ranging
if not remote:
    remote_id = None
th = 1000
while True:
    ser = serial.Serial(port=port,baudrate=28800, timeout = 1)
    #print ser

    destination_id_1 = 0x6966      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_1 = DeviceRange()
    status_1 = p.doRanging(destination_id_1, device_range_1, remote_id)
    zd_1= device_range_1[1]
    print(device_range_1[1])
    if status_1:
        list_offset = range(0, 1015, 49)
        data_length = 49
        sys.stdout.flush()
        ser.nonblocking()
        ser.flushInput()  #flush input buffer, discarding all its contents
        ser.flushOutput()
        ser.nonblocking()
        cir_buffer = Buffer([0] * 98 * len(list_offset), size=2, signed=1)
        status_cir = p.getDeviceCir(list(list_offset), data_length, cir_buffer, remote_id)
        if status_cir:
            try:
                #get real and imaginarypart of the cir buffer
                real = np.array(cir_buffer.data[0::2])
                imag = np.array(cir_buffer.data[1::2])
                # create an image of the CIR
                cira = real + 1j*imag
                #print(device_range)
                #mat[i][j] = dist
                #print(dist)
                #That plots the CIR contains in the buffer.
                #It still requires post-procesing to
                #re-align delay and received power level.
                #plt.plot((abs(cira[:-36])))
                dist_1 = device_range_1[1]-cal(cira,th)
                print(dist_1)
                #plt.show()
            except:
                print('Buffer Error on 1')
        else:
            print('error in getting cir')
    else:
        print('Ranging failed on 1')
        zd_1= device_range_1[1]
    sys.stdout.flush()
    ser.flushInput()  #flush input buffer, discarding all its contents
    ser.flushOutput()
    destination_id_2 = 0x675a      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_2 = DeviceRange()
    status_2 = p.doRanging(destination_id_2, device_range_2, remote_id)
    zd_2= device_range_2[1]
    print(device_range_2[1])
    if status_2:
        list_offset = range(0, 1015, 49)
        data_length = 49
        sys.stdout.flush()
        ser.nonblocking()
        ser.flushInput()  #flush input buffer, discarding all its contents
        ser.flushOutput()
        cir_buffer = Buffer([0] * 98 * len(list_offset), size=2, signed=1)
        status_cir = p.getDeviceCir(list(list_offset), data_length, cir_buffer, remote_id)
        if status_cir:
            try:
                import matplotlib.pyplot as plt
                import numpy as np
                #get real and imaginarypart of the cir buffer
                real = np.array(cir_buffer.data[0::2])
                imag = np.array(cir_buffer.data[1::2])
                # create an image of the CIR
                cira = real + 1j*imag
                #print(device_range)
                #mat[i][j] = dist
                #print(dist)
                #That plots the CIR contains in the buffer.
                #It still requires post-procesing to
                #re-align delay and received power level.
                #plt.plot((abs(cira[:-36])))
                dist_2 = device_range_2[1]-cal(cira,th)
                print(dist_2)
                #plt.show()
            except:
                print('Buffer error on 2')
        else:
            print('error in getting cir')
    else:
        print('Ranging failed on 2')
        zd_2= device_range_2[1]

    #destination_id_3 = 0x6a32      # network ID of the ranging destination
    destination_id_3 = 0x6758
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_3 = DeviceRange()
    status_3 = p.doRanging(destination_id_3, device_range_3, remote_id)
    zd_3= device_range_3[1]
    print(device_range_3[1])
    if status_3:
        list_offset = range(0, 1015, 49)
        data_length = 49
        sys.stdout.flush()
        ser.nonblocking()
        ser.flushInput()  #flush input buffer, discarding all its contents
        ser.flushOutput()
        cir_buffer = Buffer([0] * 98 * len(list_offset), size=2, signed=1)
        status_cir = p.getDeviceCir(list(list_offset), data_length, cir_buffer, remote_id)
        if status_cir:
            try:
                import matplotlib.pyplot as plt
                import numpy as np
                    #get real and imaginarypart of the cir buffer
                real = np.array(cir_buffer.data[0::2])
                imag = np.array(cir_buffer.data[1::2])
                    # create an image of the CIR
                cira = real + 1j*imag
                #print(device_range)
                #mat[i][j] = dist
                #print(dist)
                #That plots the CIR contains in the buffer.
                #It still requires post-procesing to
                #re-align delay and received power level.
                #plt.plot((abs(cira[:-36])))
                dist_3 = device_range_3[1]-cal(cira,th)
                print(dist_3)
                #plt.show()
            except:
                print('Buffer Error on 3')
        else:
            print('error in getting cir')
    else:
        print('Ranging failed on 3')
        zd_3= device_range_3[1]

    ser.flushInput()  #flush input buffer, discarding all its contents
    ser.flushOutput()
    destination_id_4 = 0x6932      # network ID of the ranging destination
    range_step_mm = 1000         # distance that separates the amount of LEDs lighting up.
    device_range_4 = DeviceRange()
    status_4 = p.doRanging(destination_id_4, device_range_4, remote_id)
    zd_4= device_range_4[1]
    print(device_range_4[1])
    if status_4:
        list_offset = range(0, 1015, 49)
        data_length = 49
        sys.stdout.flush()
        ser.nonblocking()
        ser.flushInput()  #flush input buffer, discarding all its contents
        ser.flushOutput()
        cir_buffer = Buffer([0] * 98 * len(list_offset), size=2, signed=1)
        status_cir = p.getDeviceCir(list(list_offset), data_length, cir_buffer, remote_id)
        if status_cir:
            try:
                import matplotlib.pyplot as plt
                import numpy as np
                #get real and imaginarypart of the cir buffer
                real = np.array(cir_buffer.data[0::2])
                imag = np.array(cir_buffer.data[1::2])
                # create an image of the CIR
                cira = real + 1j*imag
                #print(device_range)
                #mat[i][j] = dist
                #print(dist)
                #That plots the CIR contains in the buffer.
                #It still requires post-procesing to
                #re-align delay and received power level.
                #plt.plot((abs(cira[:-36])))
                dist_4 = device_range_4[1]-cal(cira,th)
                print(dist_4)
                #plt.show()
            except:
                print('Buffer Error on 4')
        else:
            print('error in getting cir')
    else:
        print('Ranging failed on 4')
        zd_4= device_range_4[1]

    # Multilateration Algorithm #
    import numpy as np

    def tri(zd, H):
        zd = zd.T
        H=H-H[0]
        tmp = zd[0]
        tmp = tmp * tmp
        zd = np.delete(zd,(0), axis=0)
        r1=np.multiply(zd,zd)
        Ha=np.delete(H,(0), axis=0)
        K=np.multiply(Ha,Ha)
        K=K.sum(axis=1)
        b= 0.5*(K-r1+tmp) 
        S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
        x_hat = S_inv*b
        return x_hat
    H = np.array([[0,6.78],[6.42,7.20],[5.4,15.52],[0,15.26]])
    H2 = np.matrix([[0,6.78],[6.42,7.20],[5.4,15.52],[0,15.26]])  
    zd = np.matrix([dist_1,dist_2,dist_3,dist_4])
    print(zd)
    x_hat = tri(zd*10**(-3),H)
    x_hat = x_hat.T+H[0]
    print(x_hat)
    zd_nlos = np.matrix([zd_1,zd_2,zd_3,zd_4])
    print(zd_nlos)
    x_nlos = tri(zd_nlos*10**(-3),H)
    x_nlos = x_nlos.T+H[0]
    print(x_nlos)
    write_ws['A1'] = 'NLOS'
    write_ws['C1'] = 'CIR'
    write_ws.cell(i+1,1,x_nlos[(0,0)])
    write_ws.cell(i+1,2,x_nlos[(0,1)])
    write_ws.cell(i+1,3,x_hat[(0,0)])
    write_ws.cell(i+1,4,x_hat[(0,1)])
    write_wb.save('/Users/swbaek/Desktop/CIR_EX_21.xlsx')
    i = i+1

2162


NameError: name 'Buffer' is not defined

In [309]:
mag = 20*np.log10(abs(cira[:-36]))
for i in range(1,1029+1):
    if(mag[i]>60):
        idx = i
        break
print(idx)

<ipython-input-309-880064d56476>:1: RuntimeWarning: divide by zero encountered in log10
  mag = 20*np.log10(abs(cira[:-36]))


IndexError: index 993 is out of bounds for axis 0 with size 993

In [ ]:
mag_arr = 20*np.log10(abs(cira[:-36]))
mag_idx=np.argmax(mag_arr)
print(mag_idx)

In [ ]:
64*3*pow(10,8)/(499.2*pow(10,6))

In [ ]:
(mag_idx-idx)*64*3*pow(10,8)/(499.2*pow(10,6))

In [ ]:
mat = np.array(40,100)

In [ ]:
print(mat)

In [ ]:
mat[0][20]

In [ ]:
pip install pypozyx

In [ ]:
pip install pyserial

In [ ]:
import serial.tools.list_ports
list(serial.tools.list_ports.comports())

In [4]:
from pypozyx import *
list_serial_ports()

/dev/cu.Bluetooth-Incoming-Port - n/a
/dev/cu.usbmodem3270345730371 - Pozyx Virtual ComPort in FS Mod


In [37]:
print(10**(-3))

0.001


In [40]:
import numpy as np

def SX(tdoa, H):
    tdoa = tdoa.T
    H=H-H[0]
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    r1=np.multiply(tdoa,tdoa)
    d=K-r1
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    a = 4-4*(tdoa.T*S_inv.T*S_inv*tdoa)
    b = 4*tdoa.T*S_inv.T*S_inv*d
    c = (-1)*(d.T*S_inv.T*S_inv*d)
    r1 = (-b+np.sqrt(pow(b,2)-4*a*c))/(2*a)
    x_hat = 0.5*S_inv*(d-2*r1[0,0]*tdoa)
    return x_hat

H = np.matrix([[7.54,0],[14.14,0],[7.54,7.21],[14.14,7.58]])

zd1 =  device_range_2[1]
zd2 =  device_range_3[1]
zd3 =  device_range[1]
zd4 =  device_range_4[1]

tdoa_col_1 = (zd2-zd1)*(10**(-3))
tdoa_col_2 = (zd3-zd1)*(10**(-3))
tdoa_col_3 = (zd4-zd1)*(10**(-3))
tdoa = np.matrix([tdoa_col_1,tdoa_col_2,tdoa_col_3])
print(tdoa)

x_hat = SX(tdoa,H)
x_hat = x_hat.T + H[0]
print(x_hat)

[[ 3.079 -3.068  1.907]]
[[7.71026389 5.05076927]]


In [129]:
print(64*3*pow(10,8)/(499.2*pow(10,6)))

38.46153846153846


In [73]:
# Multilateration Algorithm #
import numpy as np
def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat

H = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
zd = np.matrix([dist_1,dist_2,dist_3,dist_4])
print(zd)
x_hat = tri(zd*10**(-3),H)
x_hat = x_hat.T+H[0]
print(x_hat)
zd_nlos = np.matrix([zd_1,zd_2,zd_3,zd_4])
print(zd_nlos)
x_nlos = tri(zd_nlos*10**(-3),H)
x_nlos = x_nlos.T+H[0]
print(x_nlos)


[[5345.53846154 4472.84615385 5130.23076923 4232.76923077]]
[[11.06452965  4.25084282]]
[[5384 4569 5361    0]]
[[11.42025461  4.70793333]]


In [68]:
print(H[0])

[[7.54 0.  ]]


In [158]:
print(x_nlos[0,0])

9.992035714004238
